### Library Installation

In [43]:
#!pip install pandas
#!pip install beautifulsoup4  
#!pip install selenium
#!pip install requests

In [44]:
import pandas as pd


### File Importing

In [45]:
df = pd.read_csv("city_data.csv", delimiter='|')
df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,City,Population Density,Population,Working Age Population,Youth Dependency Ratio,Unemployment Rate,GDP per Capita,Days of very strong heat stress,Main Spoken Languages,Average Monthly Salary,Avgerage Rent Price,Average Cost of Living,Average Price Groceries,Last Data Update
1,"Vienna, Austria",310,2983513,2018818,20.1,10.2,55770,3,"German, English, Turkish, Serbian",2500,1050,2061,340,2024-06-15 00:00:00
2,"Salzburg, Austria",243,375489,250472,20.3,3,66689,0,German,3200,1100,2186,NaN,2023-11-03 00:00:00
3,"Brussels, Belgium",681,3284548,2137425,27.5,10.7,62500,3,"French, Dutch, Arabic, English",3350,1200,1900,NaN,2023-04-22 00:00:00
4,"Antwerp, Belgium",928,1139663,723396,27.7,6.2,57595,3,"Dutch, French, Arabic",2609,900,1953,NaN,2024-08-09 00:00:00


### Header Correction

In [46]:
new_header = df.iloc[0]

df.columns = new_header
df = df[1:]

df.head()

,City,Population Density,Population,Working Age Population,Youth Dependency Ratio,Unemployment Rate,GDP per Capita,Days of very strong heat stress,Main Spoken Languages,Average Monthly Salary,Avgerage Rent Price,Average Cost of Living,Average Price Groceries,Last Data Update
1,"Vienna, Austria",310,2983513,2018818,20.1,10.2,55770,3,"German, English, Turkish, Serbian",2500,1050,2061,340,2024-06-15 00:00:00
2,"Salzburg, Austria",243,375489,250472,20.3,3,66689,0,German,3200,1100,2186,NaN,2023-11-03 00:00:00
3,"Brussels, Belgium",681,3284548,2137425,27.5,10.7,62500,3,"French, Dutch, Arabic, English",3350,1200,1900,NaN,2023-04-22 00:00:00
4,"Antwerp, Belgium",928,1139663,723396,27.7,6.2,57595,3,"Dutch, French, Arabic",2609,900,1953,NaN,2024-08-09 00:00:00
5,"Gent, Belgium",552,645813,417832,24.8,NaN,53311,2,"Dutch, French",2400,827,1200,120,2023-07-17 00:00:00


## Data Wrangling and Analysis

1. How did you handle missing values and duplicate records in the dataset? Justify your approach.


2. 
    
        a) Which country appears most frequently in the dataset? How many cities are associated with it?

First we need to separate the city name from the country to which it belongs

The problem is that not all pairs in the City collumn have the same formatting.

In [47]:
temp = df.copy()
print(temp.iloc[2,0])
print(temp.iloc[12,0])
print(temp.iloc[15,0])
print(temp.iloc[38,0])

Brussels, Belgium
Lemesos;Cyprus
Berlin. Germany
Lyon,  France


So we will correct it by replacing the formatting errors such as ';', '.' and double spaces with the same ', '.

In [48]:
corrected = []
for city in temp.City:
    if ';' in city:
        corrected.append(city.replace(';', ', '))
    elif '.'in city:
        corrected.append(city.replace('.', ','))
    elif '  ' in city:
        corrected.append(city.replace('  ', ' '))
    else:
        corrected.append(city)

temp.City = corrected

print(temp.iloc[2,0])
print(temp.iloc[12,0])
print(temp.iloc[15,0])
print(temp.iloc[38,0])

Brussels, Belgium
Lemesos, Cyprus
Berlin, Germany
Lyon, France


Another problem is that in row 45 the pair is in 'country, city' format instead of 'city, country', we can solve it by manually correcting it.

In [49]:
temp.iloc[45]

0
City                                    Greece, Athens
Population Density                                1829
Population                                     3530371
Working Age Population                         2287174
Youth Dependency Ratio                              22
Unemployment Rate                                 17.2
GDP per Capita                                   38580
Days of very strong heat stress                     17
Main Spoken Languages                   Greek, English
Average Monthly Salary                            1050
Avgerage Rent Price                                600
Average Cost of Living                            1200
Average Price Groceries                            NaN
Last Data Update                   2024-07-16 00:00:00
Name: 46, dtype: object

In [50]:
temp.iloc[45, 0] = 'Athens, Greece'

temp.iloc[45]

0
City                                    Athens, Greece
Population Density                                1829
Population                                     3530371
Working Age Population                         2287174
Youth Dependency Ratio                              22
Unemployment Rate                                 17.2
GDP per Capita                                   38580
Days of very strong heat stress                     17
Main Spoken Languages                   Greek, English
Average Monthly Salary                            1050
Avgerage Rent Price                                600
Average Cost of Living                            1200
Average Price Groceries                            NaN
Last Data Update                   2024-07-16 00:00:00
Name: 46, dtype: object

Now that all pairs have consistent formatting we can split the pairs in CityName and Country.

In [51]:
#Extract the country

pairs = [city.split(', ') for city in temp.City]

cities = [pair[0] for pair in pairs]
countries = [pair[1] for pair in pairs]

print(cities)
print(countries)

['Vienna', 'Salzburg', 'Brussels', 'Antwerp', 'Gent', 'Bruges', 'Sofia', 'Dobrich', 'Zurich', 'Geneva', 'Basel', 'Lefkosia', 'Lemesos', 'Prague', 'Ostrava', 'Berlin', 'Berlin', 'Hamburg', 'Munich', 'Cologne', 'Frankfurt am Main', 'Stuttgart', 'Leipzig', 'Dresden', 'Dusseldorf', 'Hanover', 'Copenhagen', 'Odense', 'Madrid', 'Barcelona', 'Valencia', 'Seville', 'Malaga', 'Malaga', 'Tallinn', 'Helsinki', 'Tampere', 'Paris', 'Lyon', 'Toulouse', 'London', 'Leeds', 'Glasgow', 'Liverpool', 'Edinburgh', 'Athens', 'Thessaloniki', 'Zagreb', 'Split', 'Budapest', 'Miskolc', 'Debrecen', 'Dublin', 'Cork', 'Rome', 'Milan', 'Naples', 'Turin', 'Florence', 'Venice', 'Luxembourg', 'Riga', 'Malta', 'The Hague', 'Amsterdam', 'Rotterdam', 'Utrecht', 'Eindhoven', 'Oslo', 'Bergen', 'Stavanger', 'Warsaw', 'Lodz', 'Cracow', 'Lisbon', 'Porto', 'Braga', 'Coimbra', 'Giroc', 'Bratislava', 'Ljubljana', 'Stockholm', 'Gothenburg', 'Malmo', 'Ankara', 'Adana']
['Austria', 'Austria', 'Belgium', 'Belgium', 'Belgium', 'Belgi

Now that we have the cities and country lists we can add them to the dataframe.

In [52]:
temp['CityName'] = cities
temp['Country'] = countries

temp.loc[:,['City', 'CityName', 'Country']]

,City,CityName,Country
1,"Vienna, Austria",Vienna,Austria
2,"Salzburg, Austria",Salzburg,Austria
3,"Brussels, Belgium",Brussels,Belgium
4,"Antwerp, Belgium",Antwerp,Belgium
5,"Gent, Belgium",Gent,Belgium
...,...,...,...
82,"Stockholm, Sweden",Stockholm,Sweden
83,"Gothenburg, Sweden",Gothenburg,Sweden
84,"Malmo, Sweden",Malmo,Sweden
85,"Ankara, Turkiye",Ankara,Turkiye


As we can see anothor problem arises, there are cities that are duplicated in the dataset.

In [53]:
temp[temp.duplicated(keep=False)]

,City,Population Density,Population,Working Age Population,Youth Dependency Ratio,Unemployment Rate,GDP per Capita,Days of very strong heat stress,Main Spoken Languages,Average Monthly Salary,Avgerage Rent Price,Average Cost of Living,Average Price Groceries,Last Data Update,CityName,Country
16,"Berlin, Germany",304,5303922,3481212,21.3,4.7,46548,3,"German, Turkish, Arabic, English",3200,1220,2200,NaN,2023-06-29 00:00:00,Berlin,Germany
17,"Berlin, Germany",304,5303922,3481212,21.3,4.7,46548,3,"German, Turkish, Arabic, English",3200,1220,2200,NaN,2023-06-29 00:00:00,Berlin,Germany
33,"Malaga, Spain",571,869096,585608,23.5,17.7,27694,0,"Spanish, English",2200,1312,1400,NaN,2023-11-27 00:00:00,Malaga,Spain
34,"Malaga, Spain",571,869096,585608,23.5,17.7,27694,0,"Spanish, English",2200,1312,1400,NaN,2023-11-27 00:00:00,Malaga,Spain


So we will drop the second occurence of each of these cities.

In [54]:
temp.drop_duplicates(inplace=True)

temp[temp.duplicated(keep=False)]

,City,Population Density,Population,Working Age Population,Youth Dependency Ratio,Unemployment Rate,GDP per Capita,Days of very strong heat stress,Main Spoken Languages,Average Monthly Salary,Avgerage Rent Price,Average Cost of Living,Average Price Groceries,Last Data Update,CityName,Country


We just need to check if there is a duplicated city name in the dataset with different data associated with it. As we can check in the following output this doesn't happen in our dataset

In [55]:
temp[temp.duplicated(subset=['CityName'])]

,City,Population Density,Population,Working Age Population,Youth Dependency Ratio,Unemployment Rate,GDP per Capita,Days of very strong heat stress,Main Spoken Languages,Average Monthly Salary,Avgerage Rent Price,Average Cost of Living,Average Price Groceries,Last Data Update,CityName,Country


Now we can make the changes to our DataFrame permanent by applying them to the original DataFrame.

In [56]:
df = temp

Finally we can count the name of rows (Cities) that have the same value in the 'Country' collumn.

In [57]:
df['Country'].value_counts()

Country
Germany            10
Italy               6
United Kingdom      5
Spain               5
Netherlands         5
Belgium             4
Portugal            4
Hungary             3
Poland              3
Sweden              3
France              3
Norway              3
Switzerland         3
Czechia             2
Turkiye             2
Greece              2
Denmark             2
Finland             2
Austria             2
Bulgaria            2
Cyprus              2
Croatia             2
Ireland             2
Estonia             1
Luxembourg          1
Latvia              1
Malta               1
Romania             1
Slovenia            1
Slovak Republic     1
Name: count, dtype: int64

This way we can conclude that the most frequent country is Germany and it has 11 cities associated with it.

        b) How many cities are present in total? How many are associated with Greece?

As we solved duplicate entries in the previous item the number of cities in the dataset is the number of rows in our dataset.

In [58]:
df

,City,Population Density,Population,Working Age Population,Youth Dependency Ratio,Unemployment Rate,GDP per Capita,Days of very strong heat stress,Main Spoken Languages,Average Monthly Salary,Avgerage Rent Price,Average Cost of Living,Average Price Groceries,Last Data Update,CityName,Country
1,"Vienna, Austria",310,2983513,2018818,20.1,10.2,55770,3,"German, English, Turkish, Serbian",2500,1050,2061,340,2024-06-15 00:00:00,Vienna,Austria
2,"Salzburg, Austria",243,375489,250472,20.3,3,66689,0,German,3200,1100,2186,NaN,2023-11-03 00:00:00,Salzburg,Austria
3,"Brussels, Belgium",681,3284548,2137425,27.5,10.7,62500,3,"French, Dutch, Arabic, English",3350,1200,1900,NaN,2023-04-22 00:00:00,Brussels,Belgium
4,"Antwerp, Belgium",928,1139663,723396,27.7,6.2,57595,3,"Dutch, French, Arabic",2609,900,1953,NaN,2024-08-09 00:00:00,Antwerp,Belgium
5,"Gent, Belgium",552,645813,417832,24.8,NaN,53311,2,"Dutch, French",2400,827,1200,120,2023-07-17 00:00:00,Gent,Belgium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,"Stockholm, Sweden",334,2344124,1534225,28.5,6.2,70950,0,"Swedish, English",2700,1400,2300,NaN,2024-09-11 00:00:00,Stockholm,Sweden
83,"Gothenburg, Sweden",245,1037675,672152,28.2,6.3,49588,0,"Swedish, English",2500,1200,2100,NaN,2023-03-10 00:00:00,Gothenburg,Sweden
84,"Malmo, Sweden",368,680335,436271,29.4,9.2,44387,0,"Swedish, English",2400,1100,2000,NaN,2024-07-07 00:00:00,Malmo,Sweden
85,"Ankara, Turkiye",1922,4843511,3417691,30,14.4,38916,3,Turkish,900,450,900,309,2023-06-08 00:00:00,Ankara,Turkiye


As we can see at the bottom of the privious table there are 84 cities in our dataset.

As for the number of cities associated with Greece, there are 2 cities associated with Greece (as per the next output)

In [59]:
print(df.query('Country == "Greece"').shape[0], 'rows')
df.query('Country == "Greece"')

2 rows


,City,Population Density,Population,Working Age Population,Youth Dependency Ratio,Unemployment Rate,GDP per Capita,Days of very strong heat stress,Main Spoken Languages,Average Monthly Salary,Avgerage Rent Price,Average Cost of Living,Average Price Groceries,Last Data Update,CityName,Country
46,"Athens, Greece",1829,3530371,2287174,22,17.2,38580,17,"Greek, English",1050,600,1200,NaN,2024-07-16 00:00:00,Athens,Greece
47,"Thessaloniki, Greece",381,1050568,684564,22.2,19.9,23940,10,Greek,1000,550,1100,NaN,2023-12-21 00:00:00,Thessaloniki,Greece


        c) Which is the least spoken language in the dataset? Which are the top 3 most spoken languages?

In [60]:
df[['City', 'Main Spoken Languages']].head()

,City,Main Spoken Languages
1,"Vienna, Austria","German, English, Turkish, Serbian"
2,"Salzburg, Austria",German
3,"Brussels, Belgium","French, Dutch, Arabic, English"
4,"Antwerp, Belgium","Dutch, French, Arabic"
5,"Gent, Belgium","Dutch, French"


As we can see the `Most Spoken Language` column is formatted as a string of comma separated languages, so we will firstly need to convert it to a list of the languages spoken in that city.

In [61]:
temp = df.copy()

<span style="color:red">
    <font size="7">---REMOVE---</font>
</span>

In [62]:
temp['Main Spoken Languages'].dropna(inplace=True)

<span style="color:red">
    <font size="7">------------</font>
</span>

In [63]:
temp['Main Spoken Languages'] = temp['Main Spoken Languages'].astype(str).str.split(', ')

temp.head()

,City,Population Density,Population,Working Age Population,Youth Dependency Ratio,Unemployment Rate,GDP per Capita,Days of very strong heat stress,Main Spoken Languages,Average Monthly Salary,Avgerage Rent Price,Average Cost of Living,Average Price Groceries,Last Data Update,CityName,Country
1,"Vienna, Austria",310,2983513,2018818,20.1,10.2,55770,3,"[German, English, Turkish, Serbian]",2500,1050,2061,340,2024-06-15 00:00:00,Vienna,Austria
2,"Salzburg, Austria",243,375489,250472,20.3,3,66689,0,[German],3200,1100,2186,NaN,2023-11-03 00:00:00,Salzburg,Austria
3,"Brussels, Belgium",681,3284548,2137425,27.5,10.7,62500,3,"[French, Dutch, Arabic, English]",3350,1200,1900,NaN,2023-04-22 00:00:00,Brussels,Belgium
4,"Antwerp, Belgium",928,1139663,723396,27.7,6.2,57595,3,"[Dutch, French, Arabic]",2609,900,1953,NaN,2024-08-09 00:00:00,Antwerp,Belgium
5,"Gent, Belgium",552,645813,417832,24.8,NaN,53311,2,"[Dutch, French]",2400,827,1200,120,2023-07-17 00:00:00,Gent,Belgium


Now that we have the languages in a list, we need to explode this collumn creating a new row for each language spoken in a city.

In [64]:
exploded_languages = temp.explode('Main Spoken Languages')

exploded_languages.head()

,City,Population Density,Population,Working Age Population,Youth Dependency Ratio,Unemployment Rate,GDP per Capita,Days of very strong heat stress,Main Spoken Languages,Average Monthly Salary,Avgerage Rent Price,Average Cost of Living,Average Price Groceries,Last Data Update,CityName,Country
1,"Vienna, Austria",310,2983513,2018818,20.1,10.2,55770,3,German,2500,1050,2061,340,2024-06-15 00:00:00,Vienna,Austria
1,"Vienna, Austria",310,2983513,2018818,20.1,10.2,55770,3,English,2500,1050,2061,340,2024-06-15 00:00:00,Vienna,Austria
1,"Vienna, Austria",310,2983513,2018818,20.1,10.2,55770,3,Turkish,2500,1050,2061,340,2024-06-15 00:00:00,Vienna,Austria
1,"Vienna, Austria",310,2983513,2018818,20.1,10.2,55770,3,Serbian,2500,1050,2061,340,2024-06-15 00:00:00,Vienna,Austria
2,"Salzburg, Austria",243,375489,250472,20.3,3,66689,0,German,3200,1100,2186,NaN,2023-11-03 00:00:00,Salzburg,Austria


Now we can finally check how many times each language apears in the `exploded_languages` dataframe.

In [65]:
exploded_languages['Main Spoken Languages'].value_counts()

Main Spoken Languages
English              46
German               14
Turkish              10
French               10
Dutch                 9
Italian               6
Polish                4
Swedish               4
Greek                 4
Spanish               4
Portuguese            4
Hungarian             3
Arabic                3
Norwegian             3
Russian               3
Irish Gaelic          2
Urdu                  2
Finnish               2
Danish                2
Bulgarian             2
Czech                 2
Croatian              2
Serbian               1
  Arabic              1
Estonian              1
nan                   1
Catalan               1
Spanish;Valencian     1
Scots Gaelic          1
Scots                 1
Bengali               1
Latvian               1
Luxembourgish         1
Maltese               1
Romanian              1
Slovak                1
Slovene               1
Name: count, dtype: int64

The least spoken language is Slovene and the 3 most spoken are English, German and Turkish.

3. 

        a) Entries uploaded before April 2023 need to be updated. Which cities would require an update?


        b) How many days ago was the last update? On what day, month, and year did it occur?


4. 

        a) How are the Unemployment Rate and GDP per Capita distributed and related? What does this relationship suggest? Provide a visual representation.


        b) Which are the top 5 cities with the largest difference between the Average Monthly Salary and Average Cost of Living?
        What about the top 5 countries with the smallest average difference?
        Show these results with meaningful visualizations.


        c) Which is the best city for someone seeking:
                an average monthly salary above €1600,
                a cost of living below €900, and
                a country suitable for starting a family (with a    relatively larger youth population)?


5. What are three additional insights you find meaningful when comparing the given cities?

## Advanced Topic - Building an Interactive Map

### 1. Web Scraping

To get the exact geographical coordinates for each city we need to webscrape for each city's wikipedia entry and from there get the latitude and longitude for each of them.

#### Importing Libraries needed for web scrapping

In [ ]:
from bs4 import BeautifulSoup               # process html
from selenium import webdriver              # automate web browser interaction
from selenium.webdriver.common.by import By # specify specify how to locate elements on a web page.
import time                                 # set time delays

#### Create a browser instance

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

# The following lines of code were needed for the browser to not crash
options = Options()
options.add_argument("--headless=new")
options.add_argument("--window-size=1920,1080")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-gpu")
options.add_argument("--disable-software-rasterizer")


browser = webdriver.Chrome(options=options) # creates a new instance of a Chrome browser

url = 'https://en.wikipedia.org/wiki/Main_Page'


#### Get the city's coordinates

In [ ]:
latitudes = []
longitudes = []

for i in df['City']:
    
    browser.get(url) # opens the main wikipedia page

    searchbox = browser.find_element(By.CLASS_NAME, 'cdx-text-input__input') # finds the wikepedia search bar
    searchbox.clear() # clears all previous inputs from the search bar
    searchbox.send_keys(i) # inserts the city + country into the searchbar
    time.sleep(2)

    firstresult = browser.find_element(By.CLASS_NAME, 'cdx-menu-item__content') # finds the first suggested result
    firstresult.click() # clicks on the first suggestion
    time.sleep(2)
    
    #If the link leads to a search result page instead of a city page, we neet the first result
    page_title = browser.title
    if "Search results" in page_title:
        results = browser.find_elements(By.CSS_SELECTOR, ".mw-search-result-heading a") # finds the first result
        if results:
            results[0].click() # clicks on it
            time.sleep(2)
    
    html_source = browser.page_source # gets the page source code
    print(browser.title)
    
    soupObj = BeautifulSoup(html_source, 'html.parser') # parses the source code into a soup object

    latitude = soupObj.find(attrs={"class": 'latitude'}).text   # gets the city latitude
    longitude = soupObj.find(attrs={"class": 'longitude'}).text # gets the city latitude

    latitudes.append(latitude)   # saves the city latitude into a list
    longitudes.append(longitude) # saves the city longitude into a list

Vienna - Wikipedia
Salzburg - Wikipedia
Brussels - Wikipedia
Antwerp - Wikipedia
Ghent - Wikipedia
Bruges - Wikipedia
Sofia - Wikipedia
Dobrich - Wikipedia
Zurich - Wikipedia
Geneva - Wikipedia
Basel - Wikipedia
Nicosia - Wikipedia
Limassol - Wikipedia
Prague - Wikipedia
Ostrava - Wikipedia
Berlin - Wikipedia
Hamburg - Wikipedia
Munich - Wikipedia
Cologne - Wikipedia
Frankfurt - Wikipedia
Stuttgart - Wikipedia
Leipzig - Wikipedia
Dresden - Wikipedia
Düsseldorf - Wikipedia
Hanover - Wikipedia
Copenhagen - Wikipedia
Odense - Wikipedia
Madrid - Wikipedia
Barcelona - Wikipedia
Valencia - Wikipedia
Seville - Wikipedia
Málaga - Wikipedia
Tallinn - Wikipedia
Helsinki - Wikipedia
Tampere - Wikipedia
Paris - Wikipedia
Lyon - Wikipedia
Toulouse - Wikipedia
London - Wikipedia
Leeds - Wikipedia
Glasgow - Wikipedia
Liverpool - Wikipedia
Edinburgh - Wikipedia
Athens - Wikipedia
Thessaloniki - Wikipedia
Zagreb - Wikipedia
Split, Croatia - Wikipedia
Budapest - Wikipedia
Miskolc - Wikipedia
Debrecen - 

#### Closing browser instance

In [ ]:
browser.quit() #close the browser instance.

#### Add coordinates to dataframe corresponding them with the respective city

In [ ]:
# Add the latitude and longitude lists as collumns to the end of the data frame

df = pd.concat([df, pd.Series(latitudes, name= 'Latitude'), pd.Series(longitudes, name= 'Longitude')], axis= 1)

df.head()

,City,Population Density,Population,Working Age Population,Youth Dependency Ratio,Unemployment Rate,GDP per Capita,Days of very strong heat stress,Main Spoken Languages,Average Monthly Salary,Avgerage Rent Price,Average Cost of Living,Average Price Groceries,Last Data Update,CityName,Country,Latitude,Longitude
1,"Vienna, Austria",310,2983513,2018818,20.1,10.2,55770,3,"German, English, Turkish, Serbian",2500,1050,2061,340,2024-06-15 00:00:00,Vienna,Austria,47°48′00″N,13°02′42″E
2,"Salzburg, Austria",243,375489,250472,20.3,3,66689,0,German,3200,1100,2186,NaN,2023-11-03 00:00:00,Salzburg,Austria,50°50′48″N,04°21′09″E
3,"Brussels, Belgium",681,3284548,2137425,27.5,10.7,62500,3,"French, Dutch, Arabic, English",3350,1200,1900,NaN,2023-04-22 00:00:00,Brussels,Belgium,51°13′04″N,04°24′01″E
4,"Antwerp, Belgium",928,1139663,723396,27.7,6.2,57595,3,"Dutch, French, Arabic",2609,900,1953,NaN,2024-08-09 00:00:00,Antwerp,Belgium,51°03′13″N,03°43′31″E
5,"Gent, Belgium",552,645813,417832,24.8,NaN,53311,2,"Dutch, French",2400,827,1200,120,2023-07-17 00:00:00,Gent,Belgium,51°12′32″N,03°13′27″E


2. Interactive Map

## Data Science In Action